**TOOL BINDING**

Tool binding is a step where you register tools with a LLM so that

1) The LLM nows what tools are available

2) It knows what each tool does (via description)

3) It know what input format to be used

In [1]:
import os 
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.tools import tool

In [3]:
llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-flash",
    temperature=0.7
)

In [4]:
# Using Tool decorator
@tool
def multiply(a: int, b: int)-> int:
    """Given two numbers a and b this tool returns thre product"""
    return a*b

print(multiply.invoke({'a':3, 'b':4}))

12


In [5]:
# Using Structured tool
from langchain_core.tools import StructuredTool

def multiply2(a: int, b: int) -> int:
    """Given two numbers a and b this tool returns their product"""
    return a * b

multi = StructuredTool.from_function(
    func=multiply2,
    name="Multiplication Tool",
    description="This tool is responsible for multiplication of two numbers"
)

print(multi.invoke({"a": 2, "b": 3}))


6


**TOOL CALLING**

Tool calling is the process where the LLM decides during the conversation or task that it needs to use specific tool (function) and genertes structured output

The LLM does not actually run the tool - It just suggests the tool and the input arguments. The actual excution is handle by Lnagchain or you

In [6]:
#Tool binding
llm_with_tools = llm.bind_tools([multiply])

In [7]:
response = llm_with_tools.invoke("Hey how are you?")
response

AIMessage(content="I'm doing well, thank you for asking! How are you today?", additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []}, id='run-09615d9e-47ba-4283-8810-e82797da2ab7-0', usage_metadata={'input_tokens': 24, 'output_tokens': 17, 'total_tokens': 41, 'input_token_details': {'cache_read': 0}})

In [9]:
response = llm_with_tools.invoke("Can you multiply 2 with 100 and tell me the answer?")
print(response)

content='' additional_kwargs={'function_call': {'name': 'multiply', 'arguments': '{"a": 2.0, "b": 100.0}'}} response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-1.5-flash', 'safety_ratings': []} id='run-4adfafdf-51da-4bd5-bc6e-fd412bf6589a-0' tool_calls=[{'name': 'multiply', 'args': {'a': 2.0, 'b': 100.0}, 'id': 'eef8f8a2-97cd-4c81-adea-86c9024a158d', 'type': 'tool_call'}] usage_metadata={'input_tokens': 35, 'output_tokens': 5, 'total_tokens': 40, 'input_token_details': {'cache_read': 0}}


In [13]:
response.tool_calls

[{'name': 'multiply',
  'args': {'a': 2.0, 'b': 100.0},
  'id': 'eef8f8a2-97cd-4c81-adea-86c9024a158d',
  'type': 'tool_call'}]

In [12]:
response.tool_calls[0]["args"]

{'a': 2.0, 'b': 100.0}

In [14]:
multiply.invoke(response.tool_calls[0]["args"])

200